In [2]:
library(data.table)
library(dtplyr)
library(dplyr, warn.conflicts = FALSE)
library(lubridate)

In [3]:
meta <- fread(snakemake@input$data)
head(meta)
nrow(meta)

In [4]:
ids <- fread(snakemake@input$ids, header = F)
ids <- unlist(ids$V1)
meta <- meta %>%
    filter(taxon %in% ids) %>%
    as.data.frame()
nrow(meta)

In [5]:
lineage4focus=snakemake@params$id

In [6]:
df_focus = meta %>%
    lazy_dt() %>%
    mutate(date = date(date)) %>%
    mutate(month = lubridate::floor_date(date, "month")) %>%
    as.data.table() 
head(df_focus)

In [7]:
df_focus_per_month <- df_focus %>%
    lazy_dt() %>%
    group_by(month) %>%
    summarise(N=n()) %>%
    as.data.table()
print(df_focus_per_month)

In [8]:
max_idx <- which.max(df_focus_per_month$N) 
months_to_take <- df_focus_per_month$month[[max_idx]]

In [9]:
df_focus <- df_focus %>%
    lazy_dt() %>%
    filter(month == months_to_take) %>%
    as.data.table()
df_other <- df_focus %>%
    lazy_dt() %>%
    filter(month != months_to_take) %>%
    as.data.table()

In [10]:
ids_focus <- df_focus %>%
    select(taxon) %>%
    as.data.table()
ids_other <- df_other %>%
    select(taxon) %>%
    as.data.table()
fwrite(x = ids_focus,file = snakemake@output$ids_focus, col.names = F)
fwrite(x = ids_other,file = snakemake@output$ids_other, col.names = F)